# Optimize the prophet model to predict SPY prices

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews
!pip install textblob
!pip install schedule
!pip install time
!pip install telebot
!pip install telethon
!pip install transformers
!pip install vadersentiment
!pip install alpha_vantage

## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/huggingface/peft.git to /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-8ucfrj76
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-8ucfrj76
  Resolved https://github.com/huggingface/peft.git to commit e72a96f7cfdb330236c1b6dfe49be398d83f089e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-7qdf9a7k
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-7qdf9a7k
  Resolved https://github.com/huggingface/accelerate.git to commit 12a007d55937345aa986f5d7b1a1b6f2038465a7
  Installing build dependen

RuntimeError: This module must ONLY run as part of a Colab notebook!

In [ ]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import schedule
import time
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from GoogleNews import GoogleNews
import yfinance as yf
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout


import pandas as pd
%matplotlib inline
from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.main import check_and_make_directories
from pprint import pprint
from stable_baselines3.common.logger import configure
import sys
sys.path.append("../FinRL")

import itertools

from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

from finrl.config_tickers import DOW_30_TICKER

In [ ]:
def get_numeric_data(ticker):
  # Define the ticker symbol and time period
  #period = "3mo" #must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']"
  #interval = "1h"

  # Retrieve the historical market data
  #data = yf.download(tickers=ticker, period=period, interval=interval)
  
  START_DATE = '2009-01-01'
  END_DATE = '2024-07-01'

  df = YahooDownloader(start_date = START_DATE,
                      end_date = END_DATE,
                      ticker_list = [ticker]).fetch_data()

  df.head()
  df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
  df.sort_values(['date','tic'],ignore_index=True).head()

  fe = FeatureEngineer(
                      use_technical_indicator=True,
                      tech_indicator_list = config.INDICATORS,
                      use_turbulence=True,
                      user_defined_feature = False)

  processed = fe.preprocess_data(df)
  processed.sort_values(['date','tic'],ignore_index=True).head(1)

  return processed

ticker = 'SPY'
numeric_df = get_numeric_data(ticker)
numeric_df.tail()

In [ ]:
def prophet_forecast(df, future_days):
    # Select the necessary columns
    df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma', 'turbulence']]

    # Rename the columns to match Prophet's requirements
    df = df.rename(columns={'date': 'ds', 'close': 'y'})

    # Ensure the 'ds' column is of datetime type and remove timezone
    df['ds'] = pd.to_datetime(df['ds']).dt.tz_localize(None)

    # Create a Prophet model
    model = Prophet()

    # Add regressors to the model
    for col in df.columns:
        if col not in ['ds', 'y']:
            model.add_regressor(col)

    # Fit the model to the data
    model.fit(df)

    # Make a forecast for the next 'future_days' days
    future = model.make_future_dataframe(periods=future_days)

    # Add regressors to the future dataframe
    for col in df.columns:
        if col not in ['ds', 'y']:
            future[col] = df[col].iloc[-1]

    # Make predictions on the future dataframe
    forecast = model.predict(future)

    # Extract the forecasted close prices
    future_close_prices = forecast['yhat'].values[-future_days:]
    model.plot(forecast)
    return future_close_prices

ticker = 'SPY'
combined_df = get_numeric_data(ticker)
future_close_prices = prophet_forecast(combined_df, future_days = 30)
print("Future close prices:", future_close_prices)

In [ ]:
import optuna
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Get data
ticker = 'SPY'
df = get_numeric_data(ticker)
df = df.copy()
df.drop(['tic'], axis=1, inplace=True)

# Define the study name and store
study_name = 'prophet_optimization'
study = optuna.create_study(direction='minimize', study_name=study_name, storage='sqlite:///prophet_optimization.db', load_if_exists=True)

#optuna optimization
def objective(trial):
    feature_names = df.columns.tolist()
    feature_names.remove('date')
    feature_names.remove('close')

    features = []
    for feature in feature_names:
        if trial.suggest_categorical(feature, [True, False]):
            features.append(feature)

    # Define the hyperparameter space
    growth = trial.suggest_categorical('growth', ['linear'])
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['multiplicative'])
    changepoint_prior_scale = trial.suggest_loguniform('changepoint_prior_scale', 0.3, 0.7)
    seasonality_prior_scale = trial.suggest_loguniform('seasonality_prior_scale', 3.0, 7.0)
    holidays_prior_scale = trial.suggest_loguniform('holidays_prior_scale', 3.0, 7.0)

    # Create the Prophet model with the suggested hyperparameters
    model = Prophet(growth=growth, seasonality_mode=seasonality_mode,
                    changepoint_prior_scale=changepoint_prior_scale,
                    seasonality_prior_scale=seasonality_prior_scale,
                    holidays_prior_scale=holidays_prior_scale)

    # Prepare the data
    df_selected = df[['date', 'close'] + features]
    scaler = StandardScaler()
    df_selected[features] = scaler.fit_transform(df_selected[features])
    df_selected = df_selected.rename(columns={'date': 'ds', 'close': 'y'})

    # Add capacity values for logistic growth
    if growth == 'logistic':
        df_selected['cap'] = df_selected['y'].max()  # Set capacity to the maximum value of the time series

    # Fit the model and evaluate the metric
    model.fit(df_selected)
    df_cv = cross_validation(model, initial='730 days', period='180 days', horizon='30 days')
    score = mean_absolute_error(df_cv['y'], df_cv['yhat'])  # Use MAE as the score

    return score

# Run the optimization
study.optimize(objective, n_trials=100)

# Save the study
study.save()

# Get the best trial and print the results
best_trial = study.best_trial
best_features = [feature for feature in df.columns.tolist() if feature not in ['date', 'close'] and best_trial.params[feature]]
print("Best features:", best_features)
print("Best hyperparameters:", study.best_params)
print("Best score:", best_trial.value)